# Run Sobol sensitivity analysis

### Import packages 

In [1]:
import seaborn as sns
import itertools 
import numpy as np;
import pandas as pd;
import os,sys
import matplotlib.pyplot as plt;
import pickle

from pyswmm import Simulation;
from pyswmm import Subcatchments;
from pyswmm import Nodes;
from pyswmm import SystemStats;
from pyswmm import Simulation;
from swmmtoolbox import swmmtoolbox;
from itertools import combinations
from os.path import dirname
from math import sqrt

parent_dir = (dirname(os.getcwd()))
current_dir = os.getcwd()
sys.path.append("\\".join([current_dir, 'functions']) )

for mod in ['swmm_sims', 'sobol_sims']:
    if mod in sys.modules: 
        del sys.modules[mod]
        
import swmm_sims,sobol_sims
from swmm_sims import *   
from sobol_sims import *

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
import SALib
from SALib.analyze import sobol
from SALib.sample import saltelli
from SALib.analyze import sobol


### Set defaults for figures 

In [18]:

title_font = {'fontname':'Myriad Pro', 'size':'16', 'color':'black', 'weight':'normal',
              'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'Myriad Pro', 'size':'16'}
sub_plot = {'fontname':'Myriad Pro', 'size':'18','color':'black', 
            'horizontalalignment':'left', 'verticalalignment':'center'}

### Define Sobol parameters

In [3]:
problem = {
    'num_vars': 6,
    'names': ['k_width', 'frouted','n_perv','n_imperv','d_perv','d_imperv'],
    'bounds': [[0.01, 0.1],
               [0.,1.],
              [0.02,0.5],
              [0.01,0.02],
              [2,6],
              [0,3]]}

param_values = saltelli.sample(problem, 100)
Y = np.zeros([param_values.shape[0]])
params_df=pd.DataFrame(param_values)

with open('sobol_problem.pickle', 'wb') as handle:
    pickle.dump(param_values, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [4]:
#check length of sobol parameters
len(params_df)

1400

### Run a test model to make sure .inp file is getting updated 

In [15]:
# enter inputs here
P=2
folder='sensitivity'
dur='30min'
dstor_line = 63
perm_line = 71

#run path function to define write, read, and output file paths
write_path,read_path,out_path,run=paths(P,dur,current_dir,'sensitivity')


#step through input and print the line of the .inp file to be updated
with open(read_path) as f:
    for i, line in enumerate(f, 1):
        if i ==dstor_line:
            break
print("original .inp file         ", line) 


# pick some parameters from the param_values dictionary
n_perv = round(param_values[1][2],4)
n_imperv=round(param_values[1][3],4)
d_perv=round(param_values[1][4],4)
d_imperv =round(param_values[1][5],4)
print("new parameters -->         S1               ", n_imperv,"       ", n_perv,"       ", d_imperv,"       ",d_perv)

# set Ks
Ks=40
H_i=0
IMD=0

#run update_inp function to update .inp file with the parameters from above
update_inp(read_path,write_path,n_imperv,n_perv,d_imperv,d_perv,Ks,H_i,IMD)


#step through output and print the line of the .inp file that has the updates
with open(write_path) as f:
    for i, line in enumerate(f, 1):
        if i ==dstor_line:
            break
print("updated .inp file         ", line)           


original .inp file          S1               0.01       0.1        0          0          0          OUTLET    

new parameters -->         S1                0.0168         0.2689         2.7217         3.1211
updated .inp file          S1               0.0168       0.2689        2.7217          3.1211          0          OUTLET    



### Now run all parameters and update .inp file each time

In [25]:
folder='sensitivity'
dur='30min'

#run for five fVs
fVs=[0.1,0.3,0.5,0.7,0.9]

#run for 3 rainfall intensities
Ps=[2,5,8]

#run for three soil ksat values
Kss=[20,50,80]

Slopes=[1,5,10]

for i,p in enumerate(Ps):
    
    P = p
    print(P)
    write_path,read_path,out_path,run=paths(P,dur,current_dir,'sensitivity')

    for j,S in enumerate(Kss):
        Ks=S
        print(Ks)
        
        for k,slope in enumerate(Slopes):
            S=slope
            print(S)

            for f,fV in enumerate(fVs):
                Y = np.zeros([param_values.shape[0]])
                for m, X in enumerate(param_values):
                    
                    k_width=param_values[m][0]
                    f_routed=param_values[m][1]
                    n_perv= round(param_values[m][2],4)
                    n_imperv=round(param_values[m][3],4)
                    d_perv=round(param_values[m][4],4)
                    d_imperv=round(param_values[m][5],4)

                    A=50*100*0.0001 #A in hectares
                    TIA=(1.-fV)*A #TIA in hectares
                    A2=A-TIA #A2 in hectares
                    A3=f_routed*TIA #A3 in hectares
                    A1=TIA-A3 #A1 in hectares
                    W=k_width*(A/0.0001) #W in m

                    update_inp(read_path,write_path,n_imperv,n_perv,d_imperv,d_perv,Ks,0,0)
                    infiltration,runoff=run_sim_sens(write_path,A1,A2,A3,W,W,W,S,P)
                    total=infiltration+runoff
                    IF=infiltration/total
                    Y[m]=IF

                name="P_"+str(P)+",Ks_"+str(Ks)+",fI_"+str(round(1-fV,2))+",S_"+str(S)+".pickle"
                loc = '\\'.join([current_dir, 'out\\sensitivity\\'+str(name)])
                print(loc)
                
                with open(loc, 'wb') as handle:
                    pickle.dump(Y, handle, protocol=pickle.HIGHEST_PROTOCOL)

2
20
1
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_2,Ks_20,fI_0.9,S_1.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_2,Ks_20,fI_0.7,S_1.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_2,Ks_20,fI_0.5,S_1.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_2,Ks_20,fI_0.3,S_1.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_2,Ks_20,fI_0.1,S_1.pickle
5
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_2,Ks_20,fI_0.9,S_5.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_2,Ks_20,fI_0.7,S_5.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_2,Ks_20,fI_0.5,S_5.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_2,Ks_20,fI_0.3,S_5.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_2,Ks_20,fI_0.1,S_5.pickle
10
C:\Users\annel\One

C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_5,Ks_80,fI_0.1,S_5.pickle
10
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_5,Ks_80,fI_0.9,S_10.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_5,Ks_80,fI_0.7,S_10.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_5,Ks_80,fI_0.5,S_10.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_5,Ks_80,fI_0.3,S_10.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_5,Ks_80,fI_0.1,S_10.pickle
8
20
1
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_8,Ks_20,fI_0.9,S_1.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_8,Ks_20,fI_0.7,S_1.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_8,Ks_20,fI_0.5,S_1.pickle
C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES\out\sensitivity\P_8,Ks_20,fI_0.3,S_1.pickle
C:\Users\annel\

In [28]:
# Y = np.array(Y)

def sobol_(Y,params):
    Si = sobol.analyze(problem, Y, print_to_console=False)

    S1      = Si['S1']
    S1_conf = Si['S1_conf']
    ST      = Si['ST']
    ST_conf = Si['ST_conf']
    S2      = Si['S2']
    S2_conf = Si['S2_conf']

    k_width=params[:,0] 
    f_routed=params[:,1]
    n_perv = params[:,2]
    n_imperv =params[:,3]
    d_perv=params[:,4]
    d_imperv =params[:,5]

    
    zipped=zip(problem['names'], Si['S1'], Si['ST'], params.mean(axis=0))
    Si_df=pd.DataFrame(zipped,columns=["Name", "1st", "Total", "Mean of Input"])
    return Si_df

In [29]:
test=sobol_(Y,param_values)